In [2]:
!pip install --quiet pytorch-crf

In [3]:
import os
import random
import numpy as np

import functools

import torch
import torch.nn as nn

from torchtext import datasets
from torchtext.data import Field
from torchtext.data import BucketIterator

from torchcrf import CRF


SEED = 241

In [4]:
def seed_everything(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)

  if torch.cuda.is_available(): 
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)

In [5]:
TEXT = Field(lower=True,
             use_vocab=True,
             sequential=True,
             batch_first=True,
             include_lengths=True)

LABEL = Field(lower=True,
              use_vocab=True,
              sequential=True,
              unk_token = None,
              batch_first=True)

In [6]:
fields = [('text', TEXT), ('tags', LABEL)]

train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

downloading en-ud-v2.zip


en-ud-v2.zip: 100%|██████████| 688k/688k [00:00<00:00, 1.69MB/s]


extracting


In [7]:
TEXT.build_vocab(train_data,
                 max_size=25000,
                 vectors='glove.6B.100d')
LABEL.build_vocab(train_data)

.vector_cache/glove.6B.zip: 862MB [06:30, 2.21MB/s]                          
100%|█████████▉| 398718/400000 [00:23<00:00, 17168.85it/s]

In [8]:
batch_size = 32
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')


train_iterator = BucketIterator.splits((train_data,), batch_size=batch_size, device=device)[0]
valid_iterator = BucketIterator.splits((valid_data,), batch_size=batch_size, device=device)[0]
test_iterator = BucketIterator.splits((test_data,), batch_size=batch_size, device=device)[0]

In [37]:
class BiLSTM_CRF_Tagger(nn.Module):

  def __init__(self, vocab_size, emb_size, hidden_size, n_layers, dropout, num_tags, pad_idx, device):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=pad_idx)
    self.rnn = nn.LSTM(emb_size,
                       hidden_size, 
                       num_layers=n_layers,
                       dropout=0.3, 
                       bidirectional=True, 
                       batch_first=True)
    
    self.dropout = nn.Dropout(dropout)
    self.hidden2tag = nn.Linear(emb_size, num_tags)
    self.crf = CRF(num_tags, batch_first=True)
    self.device = device

  def _generate_mask(self, text_lens):
    bs = text_lens.size(0)
    max_seq_len = torch.max(text_lens).item()
    mask = torch.ByteTensor(bs, max_seq_len).fill_(0)
    for i in range(bs):
      mask[i, :text_lens[i]] = 1
    return mask

  def forward(self, text, text_lens, tags=None):
    text_embed = self.embedding(text)

    text_packed = nn.utils.rnn.pack_padded_sequence(text_embed, text_lens, batch_first=True, enforce_sorted=False)
    rnn_outputs, (last_hidden, cell_state) = self.rnn(text_packed)
    text_unpacked, lens_unpacked = nn.utils.rnn.pad_packed_sequence(text_packed, batch_first=True)
    last_hidden = last_hidden.permute(1, 0, 2)

    emission = self.hidden2tag(text_unpacked)
    mask = self._generate_mask(text_lens).to(self.device)

    if tags is not None:
      loss = -self.crf.forward(torch.log_softmax(emission, dim=2), tags, mask, reduction='mean')
      return loss
    else:
      prediction = self.crf.decode(emission, mask)
      return prediction

In [40]:
VOCAB_SIZE = len(TEXT.vocab)
EMB_SIZE = 100
HIDDEN_SIZE = 128
N_LAYERS = 2
DROPOUT = 0.3
NUM_TAGS = len(LABEL.vocab)
PAD_IDX = LABEL.vocab.stoi['<pad>']


model = BiLSTM_CRF_Tagger(VOCAB_SIZE, EMB_SIZE, HIDDEN_SIZE, N_LAYERS, DROPOUT, NUM_TAGS, PAD_IDX, device)
optimizer = torch.optim.Adam(model.parameters())

In [41]:
model.embedding.weight.data.copy_(TEXT.vocab.vectors)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.2634,  0.0742, -0.1081,  ..., -0.2977, -0.5655,  0.5218],
        [ 0.4244,  0.6004, -0.1528,  ...,  0.2536, -0.4969,  0.8964]])

In [42]:
model = model.to(device)

In [43]:
def train_epoch(model, iterator, optimizer):
  model.train()
  error = 0.

  for batch in iterator:
    optimizer.zero_grad()

    text, lens = batch.text
    lens = lens.cpu()
    tags = batch.tags
    
    loss = model(text, lens, tags)
    loss.backward()

    optimizer.step()
    error += loss.detach().cpu().numpy()
  return error / len(iterator)

def valid_epoch(model, iterator):
  error = 0.
  model.eval()
  with torch.no_grad():
    for batch in iterator:
      text, lens = batch.text
      lens = lens.cpu()
      tags = batch.tags
      
      loss = model(text, lens, tags)
      error += loss.detach().cpu().numpy()
  return error / len(iterator)

In [44]:
from sklearn.metrics import f1_score


def calculate_accuracy(y_true, y_pred):
  assert y_true.shape == y_pred.shape
  assert len(y_true.shape) == 1
  y_true = y_true[1:]
  y_pred = y_pred[1:]
  return (y_true == y_pred).sum() / y_true.shape[0]


def calculate_metrics(model, dataset, device):

  total_true_labels = []
  total_pred_labels = []

  for index in range(len(dataset.examples)):

    text = dataset.examples[index].text
    true_labels = dataset.examples[index].tags

    with torch.no_grad():
      tokens = text
      ids = [TEXT.vocab.stoi[token] for token in tokens]
      ids_tensor = torch.tensor([ids], device=device)
      lens = torch.tensor([len(ids)])
      prediction = model(ids_tensor, lens)
      
    # print('\t'.join(tokens))
    # print('\t'.join(true_labels))
    # print('\t'.join([LABEL.vocab.itos[p] for p in prediction[0]]))
    # print('='*20)

    total_true_labels.extend(np.array([LABEL.vocab.itos[p] for p in prediction[0]]))
    total_pred_labels.extend(np.array(true_labels))

  accurary = calculate_accuracy(np.array(total_true_labels), np.array(total_pred_labels))
  f1 = f1_score(np.array(total_true_labels), np.array(total_pred_labels), average='macro')
  print(f'Accuracy: {accurary}, F1: {f1}')

In [46]:
for epoch in range(10):
  train_error = train_epoch(model, train_iterator, optimizer)
  valid_error = valid_epoch(model, valid_iterator)

  print(f'Epoch: {epoch + 1}. Train: {train_error}, Valid: {valid_error}')
  calculate_metrics(model, test_data, device)

Epoch: 1. Train: 17.42588615904049, Valid: 7.180066464439271
Accuracy: 0.8614121772394007, F1: 0.765385346021073
Epoch: 2. Train: 6.21093812584877, Valid: 5.283478176783001
Accuracy: 0.8879502709595155, F1: 0.8115996025956886
Epoch: 3. Train: 4.365908656193286, Valid: 4.647811416595701
Accuracy: 0.8938077781319732, F1: 0.8205040273897427
Epoch: 4. Train: 3.541363316531084, Valid: 4.360866300643436
Accuracy: 0.8950430347465732, F1: 0.8212128244621087
Epoch: 5. Train: 3.0667514886174883, Valid: 4.166554924041506
Accuracy: 0.8977526299011794, F1: 0.8262088919408904
Epoch: 6. Train: 2.76078952149469, Valid: 4.057935169764927
Accuracy: 0.9002629901179471, F1: 0.8311796484604893
Epoch: 7. Train: 2.551663661185576, Valid: 4.009524122117058
Accuracy: 0.900581766018489, F1: 0.8435991671541149
Epoch: 8. Train: 2.408608538459758, Valid: 3.9787014533603
Accuracy: 0.9014185527574116, F1: 0.8468406985481658
Epoch: 9. Train: 2.304506330465784, Valid: 3.966500918070475
Accuracy: 0.9020162575709276, F1

In [47]:
# from sklearn.metrics import classification_report


# print(classification_report(np.array(total_true_labels), np.array(total_pred_labels)))

## Pruning

In [48]:
# import torch.nn.utils.prune as prune


# classifier = model.hidden2tag

# for name, param in classifier.named_parameters():
#   print(name, param)

# prune.random_unstructured(classifier, name='weight', amount=0.3)
# prune.l1_unstructured(classifier, name="bias", amount=3)

# for name, param in classifier.named_parameters():
#   print(name, param)

# print(list(classifier.named_buffers()))
# print(classifier.weight)
# print(classifier._forward_pre_hooks)

# prune.remove(classifier, 'weight')
# prune.remove(classifier, 'bias')

# print(torch.sum(model.hidden2tag.weight == 0) / model.hidden2tag.weight.nelement() * 100)
# print(calculate_metrics(model, test_data))

In [49]:
# for epoch in range(2):
#   train_error = train_epoch(model, train_iterator, optimizer)
#   valid_error = valid_epoch(model, valid_iterator)

#   print(f'Epoch: {epoch + 1}. Train: {train_error}, Valid: {valid_error}')
#   calculate_metrics(model, test_data)

## Quantization

In [50]:
model = model.cpu()

In [51]:
from torch.quantization import quantize_dynamic


quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.LSTM, nn.Linear}, dtype=torch.qint8
)

print(quantized_model)

BiLSTM_CRF_Tagger(
  (embedding): Embedding(16655, 100, padding_idx=0)
  (rnn): DynamicQuantizedLSTM(100, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (hidden2tag): DynamicQuantizedLinear(in_features=100, out_features=18, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (crf): CRF(num_tags=18)
)


In [52]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 9.200664
Size (MB): 7.317217


In [53]:
model = model.to(device)
quantized_model = quantized_model.to(device)

In [55]:
cpu_device = torch.device('cpu')
quantized_model.device = cpu_device
quantized_model.to(cpu_device)

BiLSTM_CRF_Tagger(
  (embedding): Embedding(16655, 100, padding_idx=0)
  (rnn): DynamicQuantizedLSTM(100, 128, num_layers=2, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (hidden2tag): DynamicQuantizedLinear(in_features=100, out_features=18, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
  (crf): CRF(num_tags=18)
)

In [56]:
print(calculate_metrics(model, test_data, device))
print(calculate_metrics(quantized_model, test_data, cpu_device))

Accuracy: 0.9030522792476888, F1: 0.8499894059360147
None
Accuracy: 0.9024944214217405, F1: 0.8494212786231817
None
